<a href="https://colab.research.google.com/github/dataplusplus-ai/Computer_Vision_Deep_Learning/blob/main/RadiantEarth_Starter_JW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starter Notebook - RadiantEarth Crop ID challenge (ICLR #2)

This takes you through a quick entry into the zindi contest here: https://zindi.africa/competitions/iclr-workshop-challenge-2-radiant-earth-computer-vision-for-crop-recognition

We 
- Download the data (tif files from radiant earth)
- Sample the images to get the data into an easy tabular form 
- Fit a model on this data, and experiment with grouping by field or doing pixel-wise predictions
- Make a submission to Zindi

Please give attribution if you use this notebook as inspiration for your entry :)

Good luck!

Jonathan

# Data Download

This section is essentially a clone of the proviced starter notebooks: https://github.com/radiantearth/mlhub-tutorials/tree/master/notebooks/2020%20CV4A%20Crop%20Type%20Challenge

Hopefully you won't need to run this more than once!

At the end of this section you should have all the images and labels downloaded into a folder called data.

In [ ]:
# Required libraries
import requests
from urllib.parse import urlparse
from pathlib import Path
from datetime import datetime

In [ ]:
API = 'YOUR KEY HERE'

In [ ]:
!mkdir data

In [ ]:
output_path = Path("data/")

In [ ]:
# these headers will be used in each request
headers = {
    'Authorization': f'Bearer {API}',
    'Accept':'application/json'
}

In [ ]:
def get_download_url(item, asset_key, headers):
    asset = item.get('assets', {}).get(asset_key, None)
    if asset is None:
        print(f'Asset "{asset_key}" does not exist in this item')
        return None
    r = requests.get(asset.get('href'), headers=headers, allow_redirects=False)
    return r.headers.get('Location')

def download_label(url, output_path, tileid):
    filename = urlparse(url).path.split('/')[-1]
    outpath = output_path/tileid
    outpath.mkdir(parents=True, exist_ok=True)
    
    r = requests.get(url)
    f = open(outpath/filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk:
            f.write(chunk)
    f.close()
    print(f'Downloaded {filename}')
    return 

def download_imagery(url, output_path, tileid, date):
    filename = urlparse(url).path.split('/')[-1]
    outpath = output_path/tileid/date
    outpath.mkdir(parents=True, exist_ok=True)
    
    r = requests.get(url)
    f = open(outpath/filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk:
            f.write(chunk)
    f.close()
    print(f'Downloaded {filename}')
    return

In [ ]:
# paste the id of the labels collection:
collectionId = 'ref_african_crops_kenya_02_labels'

# these optional parameters can be used to control what items are returned. 
# Here, we want to download all the items so:
limit = 100 
bounding_box = []
date_time = []

# retrieves the items and their metadata in the collection
r = requests.get(f'https://api.radiant.earth/mlhub/v1/collections/{collectionId}/items', params={'limit':limit, 'bbox':bounding_box,'datetime':date_time}, headers=headers)
collection = r.json()

In [ ]:
# retrieve list of features (in this case tiles) in the collection
for feature in collection.get('features', []):
    assets = feature.get('assets').keys()
    print("Feature", feature.get('id'), 'with the following assets', list(assets))

Feature ref_african_crops_kenya_02_tile_02_label with the following assets ['field_train_test_ids', 'field_ids', 'labels']
Feature ref_african_crops_kenya_02_tile_03_label with the following assets ['field_train_test_ids', 'field_ids', 'labels']
Feature ref_african_crops_kenya_02_tile_01_label with the following assets ['field_train_test_ids', 'field_ids', 'labels']
Feature ref_african_crops_kenya_02_tile_00_label with the following assets ['field_train_test_ids', 'field_ids', 'labels']


In [ ]:
for feature in collection.get('features', []):
    
    tileid = feature.get('id').split('tile_')[-1][:2]

    # download labels
    download_url = get_download_url(feature, 'labels', headers)
    download_label(download_url, output_path, tileid)
    
    #download field_ids
    download_url = get_download_url(feature, 'field_ids', headers)
    download_label(download_url, output_path, tileid)

In [ ]:
# paste the id of the imagery collection:
collectionId = 'ref_african_crops_kenya_02_source'

# these optional parameters can be used to control what items are returned. 
# Here, we want to download all the items so:
limit = 100 
bounding_box = []
date_time = []

# retrieves the items and their metadata in the collection
r = requests.get(f'https://api.radiant.earth/mlhub/v1/collections/{collectionId}/items', params={'limit':limit, 'bbox':bounding_box,'datetime':date_time}, headers=headers)
collection = r.json()

In [ ]:
# List assets of the items
for feature in collection.get('features', []):
    assets = feature.get('assets').keys()
    print(list(assets))
    break #all the features have the same type of assets. for simplicity we break the loop here.

In [ ]:
# This cell downloads all the multi-spectral images throughout the growing season for this competition.
# The size of data is about 1.5 GB, and download time depends on your internet connection. 
# Note that you only need to run this cell and download the data once.
i = 0
for feature in collection.get('features', []):
    assets = feature.get('assets').keys()
    tileid = feature.get('id').split('tile_')[-1][:2]
    date = datetime.strftime(datetime.strptime(feature.get('properties')['datetime'], "%Y-%m-%dT%H:%M:%SZ"), "%Y%m%d")
    for asset in assets:
        i += 1
        if i > 0: # if resuming after it failed
          download_url = get_download_url(feature, asset, headers)
          download_imagery(download_url, output_path, tileid, date)

# Opening files

Again, this is mainly code from the starter notebook shared to open the tiff files as arrays. I add in a way to view the sentinel images as RGB since that's pretty cool, but it isn't actually useful for our purposes.

In [ ]:
!pip install tifffile

In [ ]:
# Required libraries
import tifffile as tiff
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def load_file(fp):
    """Takes a PosixPath object or string filepath
    and returns np array"""
    
    return tiff.imread(fp.__str__())

In [ ]:
# List of dates that an observation from Sentinel-2 is provided in the training dataset
dates = [datetime.datetime(2019, 6, 6, 8, 10, 7),
         datetime.datetime(2019, 7, 1, 8, 10, 4),
         datetime.datetime(2019, 7, 6, 8, 10, 8),
         datetime.datetime(2019, 7, 11, 8, 10, 4),
         datetime.datetime(2019, 7, 21, 8, 10, 4),
         datetime.datetime(2019, 8, 5, 8, 10, 7),
         datetime.datetime(2019, 8, 15, 8, 10, 6),
         datetime.datetime(2019, 8, 25, 8, 10, 4),
         datetime.datetime(2019, 9, 9, 8, 9, 58),
         datetime.datetime(2019, 9, 19, 8, 9, 59),
         datetime.datetime(2019, 9, 24, 8, 9, 59),
         datetime.datetime(2019, 10, 4, 8, 10),
         datetime.datetime(2019, 11, 3, 8, 10)]

In [ ]:
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'CLD']

In [ ]:
# Sample file to load:
file_name = "data/00/20190825/0_B03_20190825.tif"
band_data = load_file(file_name)

In [ ]:
fig = plt.figure(figsize=(7, 7))
plt.imshow(band_data, vmin=0, vmax=0.15)

In [ ]:
# Quick way to see an RGB image. Can mess with the scaling factor to change brightness (3 in this example)

import numpy as np
def load_rgb(tile, date):

  r = load_file(f"data/{tile}/{date}/{tile[1]}_B04_{date}.tif")
  g = load_file(f"data/{tile}/{date}/{tile[1]}_B03_{date}.tif")
  b = load_file(f"data/{tile}/{date}/{tile[1]}_B02_{date}.tif")
  arr = np.dstack((r, g, b))
  print(max(g.flatten()))
  return arr

fig, ax = plt.subplots(figsize=(12, 18))
ax.imshow(load_rgb('01', '20190825')*3)
plt.tight_layout()

# Getting data for each pixel in fields

This is where we start massaging the data into a format we can use. I read in the labels, find the pixel locations of all the fields (most are only a few pixels in size) and store the pixel locations, the field ID and the label.

Then we loop through al the images, sampling the different image bands to get the values for each pixel in each field. This isn't a very clean way to do this, but it works and doesn't take long to run in this case :)

In [ ]:
# Not super efficient but  ¯\_(ツ)_/¯
import pandas as pd

row_locs = []
col_locs = []
field_ids = []
labels = []
tiles = []
for tile in range(4):
  fids = f'/content/data/0{tile}/{tile}_field_id.tif'
  labs = f'/content/data/0{tile}/{tile}_label.tif'
  fid_arr = load_file(fids)
  lab_arr = load_file(labs)
  for row in range(len(fid_arr)):
    for col in range(len(fid_arr[0])):
      if fid_arr[row][col] != 0:
        row_locs.append(row)
        col_locs.append(col)
        field_ids.append(fid_arr[row][col])
        labels.append(lab_arr[row][col])
        tiles.append(tile)

df = pd.DataFrame({
    'fid':field_ids,
    'label':labels,
    'row_loc': row_locs,
    'col_loc':col_locs,
    'tile':tiles
})

print(df.shape)
print(df.groupby('fid').count().shape)
df.head()

In [ ]:
# Sample the bands at different dates as columns in our new dataframe
col_names = []
col_values = []

for tile in range(4): # 1) For each tile
  print('Tile: ', tile)
  for d in dates: # 2) For each date
    print(str(d))
    d = ''.join(str(d.date()).split('-')) # Nice date string
    t = '0'+str(tile)
    for b in bands: # 3) For each band
      col_name = d+'_'+b
      
      if tile == 0:
        # If the column doesn't exist, create it and populate with 0s
        df[col_name] = 0

      # Load im
      im = load_file(f"data/{t}/{d}/{t[1]}_{b}_{d}.tif")
      
      # Going four levels deep. Each second on the outside is four weeks in this loop
      # If we die here, there's no waking up.....
      vals = []
      for row, col in df.loc[df.tile == tile][['row_loc', 'col_loc']].values: # 4) For each location of a pixel in a field
        vals.append(im[row][col])
      df.loc[df.tile == tile, col_name] = vals
df.head()


In [ ]:
df.to_csv('sampled_data.csv', index=False)

In [ ]:
!cp 'sampled_data.csv' 'drive/My Drive/' # You can mount drive and save like this, or just download sampled_data.csv and use that later.

# Trying a few simple models

Now that we have our data in a nice tabular data format, we can try some simple models. I'll restart the runtime here and load in the data we saved earlier. If you have already done the first two steps, you can just upload sampled_data.csv and start from here.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [ ]:
# Load the data
df = pd.read_csv('sampled_data.csv')
print(df.shape)
df.head()

(67557, 174)


,fid,label,row_loc,col_loc,tile,20190606_B01,20190606_B02,20190606_B03,20190606_B04,20190606_B05,20190606_B06,20190606_B07,20190606_B08,20190606_B8A,20190606_B09,20190606_B11,20190606_B12,20190606_CLD,20190701_B01,20190701_B02,20190701_B03,20190701_B04,20190701_B05,20190701_B06,20190701_B07,20190701_B08,20190701_B8A,20190701_B09,20190701_B11,20190701_B12,20190701_CLD,20190706_B01,20190706_B02,20190706_B03,20190706_B04,20190706_B05,20190706_B06,20190706_B07,20190706_B08,20190706_B8A,...,20190919_CLD,20190924_B01,20190924_B02,20190924_B03,20190924_B04,20190924_B05,20190924_B06,20190924_B07,20190924_B08,20190924_B8A,20190924_B09,20190924_B11,20190924_B12,20190924_CLD,20191004_B01,20191004_B02,20191004_B03,20191004_B04,20191004_B05,20191004_B06,20191004_B07,20191004_B08,20191004_B8A,20191004_B09,20191004_B11,20191004_B12,20191004_CLD,20191103_B01,20191103_B02,20191103_B03,20191103_B04,20191103_B05,20191103_B06,20191103_B07,20191103_B08,20191103_B8A,20191103_B09,20191103_B11,20191103_B12,20191103_CLD
0,2928,4,214,1278,0,0.1995,0.2024,0.1782,0.1898,0.2462,0.3577,0.4018,0.3846,0.4149,0.778,0.3034,0.2589,5.0,0.036,0.0409,0.0706,0.0552,0.1096,0.2826,0.3318,0.3099,0.3572,0.3578,0.2325,0.1307,0.0,0.0361,0.0446,0.0720,0.0588,0.1049,0.2644,0.3296,0.3253,0.3519,...,0.0,0.0419,0.0576,0.0913,0.0941,0.1455,0.2732,0.3229,0.2989,0.3538,0.3444,0.2957,0.1995,0.0,0.049,0.0586,0.0898,0.0711,0.1279,0.2889,0.3427,0.3218,0.3657,0.3573,0.2620,0.1538,0.0,0.0372,0.0536,0.0874,0.0669,0.1239,0.2660,0.3214,0.3204,0.3515,0.361,0.2362,0.1347,0.0
1,2928,4,214,1279,0,0.1995,0.1942,0.1848,0.1846,0.2462,0.3577,0.4018,0.3859,0.4149,0.778,0.3034,0.2589,5.0,0.036,0.0365,0.0682,0.0457,0.1096,0.2826,0.3318,0.3242,0.3572,0.3578,0.2325,0.1307,0.0,0.0361,0.0393,0.0575,0.0419,0.1049,0.2644,0.3296,0.3127,0.3519,...,0.0,0.0419,0.0479,0.0901,0.0784,0.1455,0.2732,0.3229,0.3131,0.3538,0.3444,0.2957,0.1995,0.0,0.049,0.0551,0.0854,0.0673,0.1279,0.2889,0.3427,0.3172,0.3657,0.3573,0.2620,0.1538,0.0,0.0372,0.0457,0.0705,0.0603,0.1239,0.2660,0.3214,0.3296,0.3515,0.361,0.2362,0.1347,0.0
2,2928,4,214,1280,0,0.1995,0.1868,0.1930,0.1860,0.2558,0.3590,0.3982,0.3861,0.4101,0.778,0.3041,0.2560,13.0,0.036,0.0368,0.0504,0.0458,0.1134,0.2615,0.3143,0.3272,0.3525,0.3578,0.2358,0.1311,0.0,0.0361,0.0524,0.0665,0.0637,0.1308,0.2669,0.3313,0.3185,0.3595,...,0.0,0.0419,0.0730,0.0937,0.1028,0.1490,0.2755,0.3264,0.3222,0.3491,0.3444,0.3132,0.2175,0.0,0.049,0.0605,0.0871,0.0802,0.1479,0.2784,0.3277,0.3220,0.3555,0.3573,0.2705,0.1670,0.0,0.0372,0.0565,0.0852,0.0631,0.1259,0.2579,0.3131,0.3459,0.3307,0.361,0.2304,0.1471,0.0
3,2928,4,214,1281,0,0.1995,0.1962,0.1978,0.1954,0.2558,0.3590,0.3982,0.3869,0.4101,0.778,0.3041,0.2560,13.0,0.036,0.0675,0.0881,0.0850,0.1134,0.2615,0.3143,0.2966,0.3525,0.3578,0.2358,0.1311,0.0,0.0361,0.0792,0.1056,0.1036,0.1308,0.2669,0.3313,0.3198,0.3595,...,0.0,0.0419,0.0937,0.1232,0.1358,0.1490,0.2755,0.3264,0.3106,0.3491,0.3444,0.3132,0.2175,0.0,0.049,0.0938,0.1118,0.1202,0.1479,0.2784,0.3277,0.2849,0.3555,0.3573,0.2705,0.1670,0.0,0.0372,0.1022,0.1338,0.1214,0.1259,0.2579,0.3131,0.2861,0.3307,0.361,0.2304,0.1471,0.0
4,2928,4,214,1282,0,0.1995,0.1976,0.1914,0.1956,0.2571,0.3662,0.4005,0.3796,0.4073,0.778,0.3086,0.2573,9.0,0.036,0.0891,0.1258,0.1208,0.1238,0.2837,0.3248,0.3074,0.3602,0.3578,0.2392,0.1517,0.0,0.0361,0.0673,0.1024,0.0872,0.1227,0.2783,0.3413,0.3400,0.3684,...,0.0,0.0419,0.0791,0.1192,0.1248,0.1694,0.2821,0.3247,0.3096,0.3440,0.3444,0.3087,0.2222,0.0,0.049,0.0994,0.1274,0.1090,0.1530,0.2940,0.3447,0.3229,0.3567,0.3573,0.2768,0.1726,0.0,0.0372,0.1164,0.1238,0.1278,0.1385,0.2726,0.3239,0.3286,0.3446,0.361,0.2323,0.1355,0.0


In [ ]:
# Split into train and test sets
train = df.loc[df.label != 0]
test =  df.loc[df.label == 0]
train.shape, test.shape

((47791, 174), (19766, 174))

In [ ]:
# Split train into train and val, so that we can score our models locally (test is the test set)

# Splitting on field ID since we want to predict for unseen FIELDS
val_field_ids = train.groupby('fid').mean().reset_index()['fid'].sample(frac=0.3).values
tr = train.loc[~train.fid.isin(val_field_ids)].copy()
val = train.loc[train.fid.isin(val_field_ids)].copy()

# Split into X and Y for modelling
X_train, y_train = tr[tr.columns[5:]], tr['label']
X_val, y_val = val[val.columns[5:]], val['label']

### We can predict on pixels and then combine, or group into fields and then predict. Let's try both and compare!

In [ ]:
# Predicting on pixels

# Fit model (takes a few minutes since we have plenty of rows)
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train.fillna(0), y_train)

# Get predicted probabilities
preds = model.predict_proba(X_val.fillna(0))

# Add to val dataframe as columns
for i in range(7):
  val[str(i+1)] = preds[:,i]

# Get 'true' vals as columns in val
for c in range(1, 8):
  val['true'+str(c)] = (val['label'] == c).astype(int)

pred_cols = [str(i) for i in range(1, 8)]
true_cols = ['true'+str(i) for i in range(1, 8)]

# Score
print('Pixel score:', log_loss(val[true_cols], val[pred_cols])) # Note this isn't the score you'd get for a submission!!
print('Field score:', log_loss(val.groupby('fid').mean()[true_cols], val.groupby('fid').mean()[pred_cols])) # This is what we'll compare to

# Inspect visually
val[['label']+true_cols+pred_cols].head()

Pixel score: 1.256923197055142
Field score: 1.2995584366836086


,label,true1,true2,true3,true4,true5,true6,true7,1,2,3,4,5,6,7
0,4,0,0,0,1,0,0,0,0.600,0.130,0.008,0.168,0.042,0.034,0.018
1,4,0,0,0,1,0,0,0,0.554,0.160,0.010,0.198,0.030,0.034,0.014
2,4,0,0,0,1,0,0,0,0.530,0.166,0.012,0.166,0.056,0.050,0.020
3,4,0,0,0,1,0,0,0,0.556,0.166,0.020,0.162,0.040,0.042,0.014
4,4,0,0,0,1,0,0,0,0.468,0.168,0.026,0.176,0.068,0.074,0.020


In [ ]:
# Predicting on fields (grouping first)

# Group
train_grouped = tr.groupby('fid').mean().reset_index()
val_grouped = val.groupby('fid').mean().reset_index()
X_train, y_train = train_grouped[train_grouped.columns[5:]], train_grouped['label']
X_val, y_val = val_grouped[train_grouped.columns[5:]], val_grouped['label']

# Fit model
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train.fillna(0), y_train)

# Get predicted probabilities
preds = model.predict_proba(X_val.fillna(0))

# Add to val_grouped dataframe as columns
for i in range(7):
  val_grouped[str(i+1)] = preds[:,i]

# Get 'true' vals as columns in val
for c in range(1, 8):
  val_grouped['true'+str(c)] = (val_grouped['label'] == c).astype(int)

pred_cols = [str(i) for i in range(1, 8)]
true_cols = ['true'+str(i) for i in range(1, 8)]
val_grouped[['label']+true_cols+pred_cols].head()

# Already grouped, but just to double check:
print('Field score:', log_loss(val_grouped.groupby('fid').mean()[true_cols], val_grouped.groupby('fid').mean()[pred_cols]))

Field score: 1.2901252756706434


### Predicting on fields is much faster, and a little more accurate. But feel free to experiment :)

# Making a Submission

Let's take our favourite model, make predictions and submit on Zindi!

In [ ]:
# Group test as we did for val
test_grouped = test.groupby('fid').mean().reset_index()
preds = model.predict_proba(test_grouped[train_grouped.columns[5:]])

prob_df = pd.DataFrame({
    'Field_ID':test_grouped['fid'].values
})
for c in range(1, 8):
    prob_df['Crop_ID_'+str(c)] = preds[:,c-1]
prob_df.head()

,Field_ID,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7
0,3,0.148,0.558,0.054,0.100,0.024,0.014,0.102
1,6,0.198,0.644,0.024,0.054,0.028,0.032,0.020
2,11,0.184,0.430,0.032,0.248,0.044,0.020,0.042
3,13,0.244,0.484,0.072,0.098,0.052,0.022,0.028
4,14,0.176,0.482,0.072,0.106,0.078,0.020,0.066


In [ ]:
# Check the sample submission and compare
ss = pd.read_csv('SampleSub.csv')
ss.head()

,Field_ID,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7
0,3,0,0,0,0,0,0,0
1,6,0,0,0,0,0,0,0
2,11,0,0,0,0,0,0,0
3,13,0,0,0,0,0,0,0
4,14,0,0,0,0,0,0,0


In [ ]:
# Merge the two, to get all the required field IDs
ss = pd.merge(ss['Field_ID'], prob_df, how='left', on='Field_ID')
print(ss.isna().sum()['Crop_ID_1']) # Missing fields
# Fill in a low but non-zero val for the missing rows:
ss = ss.fillna(1/7) # There are 34 missing fields
ss.to_csv('starter_nb_submission.csv', index=False)

34


# Tips coming soon!